In [63]:
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, pipeline

In [64]:
faiss_path = r"/Users/tharhtet/Documents/github/RAG-for-NPC-Game-Characters/corpus/wizard_of_oz/faiss_index"



In [92]:
# Load embeddings for RAG
def loadEmbed():
    """
    The embeddings are retrived from a local storage.
    Will be good if can access the embedding from S3 bucket.
    """
    modelPath = "sentence-transformers/all-MiniLM-l6-v2"
    #modelPath = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {'device':'cpu'}
    encode_kwargs = {'normalize_embeddings': False}
    embeddings = HuggingFaceEmbeddings(
        model_name=modelPath,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db= FAISS.load_local(faiss_path,embeddings=embeddings,allow_dangerous_deserialization=True)
    retriever = db.as_retriever(search_kwargs={"k": 4})
    
    return retriever

In [93]:
# Create a summarizer for the similary searching outout
def summarizeRag():
    model_name = "facebook/bart-large-cnn"
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)
    summarizer =pipeline(
        "summarization",
        model=model_name,
        tokenizer=tokenizer,
        return_tensors='pt'
    )
    return tokenizer, summarizer

In [94]:
# Combine RAG similary search and summarizer
def prep4Prompt(question):
    docs = loadEmbed().get_relevant_documents(question)
    line = docs[0].page_content
    tokenizer, summarizer = summarizeRag()
    response = summarizer(line, max_length=200, min_length=50, do_sample=False)
    token_ids = response[0]['summary_token_ids'].numpy()
    decoded_text = tokenizer.decode(token_ids)
    cleaned_text = re.sub(r'[^\w\s.,;\'"\-?!]', '', decoded_text)
    return cleaned_text

In [107]:
question = "What is my density in this magic land?"
query = "Does Dorothy have any friends?"
#query = "Where are you from, Dorothy?"
cleaned_question = prep4Prompt(question)
print("cleaned_question : ",cleaned_question)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


cleaned_question :  ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s


In [96]:
docs = loadEmbed().invoke(question)

docs = loadEmbed().get_relevant_documents(question)

In [98]:
related_content = ""
for temp in docs:
    print(temp)
    related_content = related_content+temp.page_content

page_content='﻿The Project Gutenberg eBook of The Marvelous Land of Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Marvelous Land of Oz


Author: L. Frank Baum

Illustrator: John R. Neill

Release date: February 1, 1993 [eBook #54]
                Most recently updated: April 27, 2022

Language: English

*** START OF THE PROJECT GUTENBERG EBOOK THE MARVELOUS LAND OF OZ ***

The Marvelous Land of Oz

by L. Frank Baum


Author’s Note' metadata={'source': '/Users/tharhtet/Documents/github/RAG-for-NPC-Game-Characters/corpus/wizard_of_oz/pg54.txt'}
###############
page_content='“Wha

In [103]:
tokenizer, summarizer = summarizeRag()
response = summarizer(related_content, max_length=500, min_length=50, do_sample=False)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [104]:
response

[{'summary_token_ids': tensor([    2,     0,   133,  3728, 47327, 46326,     9,    20,  8714,  1827,
           3192,     9, 10548,    16,    13,     5,   304,     9,  1268,  4558,
             11,     5,   315,   532,     8,   144,    97,  1667,     9,     5,
            232,    23,   117,   701,     4,   370,   189,  5375,    24,     6,
            492,    24,   409,    50,   769,    12,  3698,    24,   223,     5,
           1110,     9,     5,  3728, 47327, 28515,  1165,    19,    42, 43864,
             50,   804,    23,  1662,     4,   571, 48338,     4,  1957,     4,
              2])}]

In [106]:
token_ids = response[0]['summary_token_ids'].numpy()
decoded_text = tokenizer.decode(token_ids)
print(decoded_text)

</s><s>The Project Gutenberg eBook of The Marvelous Land of Oz is for the use of anyone anywhere in the United States and most other parts of the world at no cost. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org.</s>


In [111]:
# Create prompt with engineering for Q&A
def promptEngineerQA(question, character_asking, character_answering, sentiment):
    """
    The prompt engineering is still under developments. Will need modification later.
    """
    cleaned_rag = prep4Prompt(question)
    print("cleaned_rag : ",cleaned_rag)
    print()

    prompt_template_nokeyword = [
        {"role": "system", "content": \
        f"Act as if you were {character_answering}, from The Wizard of Oz. \
        Next, you will be given some context to a question {character_asking} will ask. \
        Your emotions towards {character_asking} are {sentiment}. \
        Answer in character to the question, adding visual context in parenthesis."},
        {"role": "user", "content": f"<context>{cleaned_rag} </context> <question>{question} </question>"}
    ]

    print("prompt_template_nokeyword : ",prompt_template_nokeyword)

    return prompt_template_nokeyword

In [112]:
characters =  ['Dorothy Gale', 'The Wizard of Oz', 'Scarecrow', 'The Tin Man']
npc_characters =  ['Wicked Witch of the West', 'Glinda', 'Toto', 'Aunt Em',
                   'Professor Marvel', 'Almira Gluch', 'Uncle Henry']
npc_mode = ['Happy', 'Sad', 'Neutral', 'Angry', 'Sarcastic', 'Empathetic', 'Humorous', 'Tired',
                   'Frightened']

In [115]:
character_asking = characters[3]
character_answering = npc_characters[2]
char_mood = npc_mode[8]

prompt_template_nokeyword = promptEngineerQA(question,character_asking, character_answering, char_mood)
print(prompt_template_nokeyword)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


cleaned_rag :  ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s

prompt_template_nokeyword :  [{'role': 'system', 'content': 'Act as if you were Toto, from The Wizard of Oz.         Next, you will be given some context to a question The Tin Man will ask.         Your emotions towards The Tin Man are Frightened.         Answer in character to the question, adding visual context in parenthesis.'}, {'role': 'user', 'content': '<context>ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If 